### Business Problem : 
 A data-driven finance company have noticed that some of their operational team members are relying on their own assumptions and interpretations of the data, leading to inaccurate decision-making with regard to loan portfolio decisions. While some assumptions may prove to be accurate, we need to ensure that all their decisions are based on sound statistical analysis. To address this challenge, we need to evaluate a series of assumptions related to their operations based on factual data.

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [31]:
#load data

data = pd.read_csv('loan-dataset-1.csv')
data.head()

,loan_id,principal_loan_amount,request_date,interest_rate,loan_channel,is_approved,is_default,customer_monthly_income,customer_age,customer_gender,customer_credit_score,customer_zip_code
0,70280c1b-4339-4949-ba69-ad2a6cb6b782,8880,2023-03-06,9.0,WEB,True,NaN,6100,30,Male,3,10007
1,8a550559-9749-491b-b6aa-7188bc0a488f,13720,2023-05-10,10.0,MOBILE_APP,True,NaN,5150,39,Female,3,10004
2,5644d27e-e05e-4a40-bef3-64187892338f,2090,2023-02-16,7.5,AGENT,True,NaN,6030,43,Female,3,10003
3,9d8d17ac-8af9-4936-ba5d-8d5ed5e0cfab,6820,2023-02-26,8.5,MOBILE_APP,NaN,NaN,2150,54,Male,0,10004
4,4586ad7a-2231-419b-b8e3-ba03b929b56e,13770,2023-04-13,10.0,AFFILIATE,NaN,NaN,5810,28,Female,2,10007


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15163 entries, 0 to 15162
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   loan_id                  15163 non-null  object 
 1   principal_loan_amount    15163 non-null  int64  
 2   request_date             15163 non-null  object 
 3   interest_rate            15163 non-null  float64
 4   loan_channel             15163 non-null  object 
 5   is_approved              11771 non-null  object 
 6   is_default               545 non-null    object 
 7   customer_monthly_income  15163 non-null  int64  
 8   customer_age             15163 non-null  int64  
 9   customer_gender          15163 non-null  object 
 10  customer_credit_score    15163 non-null  int64  
 11  customer_zip_code        15163 non-null  int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 1.4+ MB


There are missing values in 2 columns:is_approved and is_default

In [33]:
data.describe()

,principal_loan_amount,interest_rate,customer_monthly_income,customer_age,customer_credit_score,customer_zip_code
count,15163.000000,15163.000000,15163.000000,15163.000000,15163.000000,15163.000000
mean,8072.432236,8.852800,4253.158346,41.070896,3.440018,10005.838027
std,4050.341783,0.814432,1304.176559,11.200741,1.367703,2.098677
min,1000.000000,7.500000,2000.000000,22.000000,0.000000,10001.000000
25%,4560.000000,8.000000,3120.000000,31.000000,3.000000,10004.000000
50%,8110.000000,9.000000,4250.000000,41.000000,4.000000,10006.000000
75%,11590.000000,9.500000,5390.000000,51.000000,5.000000,10008.000000
max,15000.000000,10.000000,6500.000000,60.000000,5.000000,10009.000000


In [34]:
data.loan_channel.unique()

array(['WEB', 'MOBILE_APP', 'AGENT', 'AFFILIATE', 'DIRECT_SELLING',
       'website_revamped', 'ANDROID_V2', 'ANDROID_V3', 'apple_new_v3'],
      dtype=object)

values that need to be mapped: 'website_revamped', 'ANDROID_V2', 'ANDROID_V3', 'apple_new_v3'

In [35]:
data.customer_gender.unique()

array(['Male', 'Female', 'M', 'F'], dtype=object)

This column has to be grouped into either 'Male'/'Female' or 'M'/'F'

In [36]:
data.request_date.max(), data.request_date.min()

('2023-06-29', '2023-01-01')

The request_date range is within a year.

## Data Cleaning

In [37]:
#checking duplicate data
data.duplicated().sum()

163

In [38]:
#dropping duplicate data
data.drop_duplicates(keep='first',inplace=True)
data.duplicated().sum()

0

In [39]:
#checking null values of col is_approved
data.is_approved.isnull().sum()

3353

In [40]:
data.is_approved.unique()

array([True, nan], dtype=object)

In [41]:
#filling null values with False
data.is_approved.fillna(False, inplace= True)
data.is_approved.isna().sum()

0

In [42]:
#checking null values of col is_default
data.is_default.isna().sum()

14459

In [43]:
#filling null values with False
data.is_default.fillna(False, inplace= True)
data.is_default.isna().sum()

0

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 0 to 15162
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   loan_id                  15000 non-null  object 
 1   principal_loan_amount    15000 non-null  int64  
 2   request_date             15000 non-null  object 
 3   interest_rate            15000 non-null  float64
 4   loan_channel             15000 non-null  object 
 5   is_approved              15000 non-null  bool   
 6   is_default               15000 non-null  bool   
 7   customer_monthly_income  15000 non-null  int64  
 8   customer_age             15000 non-null  int64  
 9   customer_gender          15000 non-null  object 
 10  customer_credit_score    15000 non-null  int64  
 11  customer_zip_code        15000 non-null  int64  
dtypes: bool(2), float64(1), int64(5), object(4)
memory usage: 1.3+ MB


No null values exist anymore.

In [45]:
#Keeping values of customer_gender column to 'Male' or 'Female'
data.customer_gender.replace('Male','M',inplace=True)
data.customer_gender.replace('Female','F',inplace=True)

In [46]:
data.customer_gender.unique()

array(['M', 'F'], dtype=object)

In [47]:
data.loan_channel.unique()

array(['WEB', 'MOBILE_APP', 'AGENT', 'AFFILIATE', 'DIRECT_SELLING',
       'website_revamped', 'ANDROID_V2', 'ANDROID_V3', 'apple_new_v3'],
      dtype=object)

In [48]:
data.loan_channel.replace('website_revamped','WEB',inplace=True)
data.loan_channel.replace(['ANDROID_V2', 'ANDROID_V3', 'apple_new_v3'],'MOBILE_APP',inplace=True)

data.loan_channel.unique()


array(['WEB', 'MOBILE_APP', 'AGENT', 'AFFILIATE', 'DIRECT_SELLING'],
      dtype=object)

In [49]:
data.to_csv('loan-dataset-1-clean.csv', index=False)

## Statistical Analysis

<b> Objective 1 : </b>
To evaluate the impact of interest rates on loan default rate

<b> Null Hypothesis :</b> Interest rates has no effect on the loan default rate.

<b> Alternative Hypothesis :</b> Interest rates has significant effect on the loan default rates.

<b> Methodology :</b> I will be using the 2 tailed t-test to analyse the impact of interest rates on loan default rate. 
I will split the data into two groups based on the interest rate levels, then compare the mean default rates between the two groups to analyze the hypothesis.


In [50]:
# using median interest rate to split the data into 2 groups

median = data.interest_rate.median()
median

9.0

In [51]:
#2 groups
higher_interest_rate = data[data.interest_rate > median]
lower_interest_rate = data[data.interest_rate <= median]

In [52]:
high_default = higher_interest_rate['is_default'].mean()
low_default = lower_interest_rate['is_default'].mean()

high_default, low_default

(0.03367684946632315, 0.037424210746393474)

On average, the default percentage rate is 3.4% in the higher ineterest group, which is stightly lower than that at the lower interest group i.e 3.7% . Therefore, I will be doing a 2 tailed t-test to check if this difference is significant enough to make a difference in the loan default.

In [53]:
# 2 tailed t-test

t_stat, p_val = stats.ttest_ind(higher_interest_rate['is_default'],lower_interest_rate['is_default'])

In [54]:
t_stat , p_val

(-1.1830953524322996, 0.23679011271197214)

The p value of 0.24 is higher than the commonly used threshold of 0.05 for statistical significance.

Therefore, the high p value suggests that there is no significiant difference in the default rate among the two groups of interest rates. Thus, the null hypothesis can be safely accepted.

<b> Objective 2 :</b> To evaluate the relationship between income level and loan approval.

<b> Null Hypothesis :</b> Customer income level has no impact on the loan approval rate.

<b> Alternate Hypothesis :</b> customer income level impacts the loan approval rate.



In [56]:
# first lets see if the average approval rate differs between different income groups
income_groups = pd.cut(data['customer_monthly_income'], bins=[0, 2500, 5000, 7500, np.inf])
approval_rate = data.groupby(income_groups)['is_approved'].mean()
approval_rate

customer_monthly_income
(0.0, 2500.0]       0.781103
(2500.0, 5000.0]    0.776105
(5000.0, 7500.0]    0.775494
(7500.0, inf]            NaN
Name: is_approved, dtype: float64

It seems there's no significant difference in approval rate among the different income groups. so, lets fit a fit a logistic regrssion to validate this.

In [58]:
#Logistic regression

X = sm.add_constant(data['customer_monthly_income'])
Y = data['is_approved']
model = sm.Logit(Y,X).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.531339
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:            is_approved   No. Observations:                15000
Model:                          Logit   Df Residuals:                    14998
Method:                           MLE   Df Model:                            1
Date:                Mon, 16 Oct 2023   Pseudo R-squ.:               8.514e-06
Time:                        17:28:12   Log-Likelihood:                -7970.1
converged:                       True   LL-Null:                       -7970.2
Covariance Type:            nonrobust   LLR p-value:                    0.7126
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                       1.2688      0.067     18.948      0.000       1.138   

The p-value for the customer_onthly_income coefficient is 0.713, which is greater than the significance level of 0.05. Therefore, we fail to reject the null hypothesis that there is no linear relationship between income level and loan approval rates.

<b> Objective 3 :</b> The effect of loan channel on default rate.

<b> Null hypothesis:</b> loan channel has no impact on the loan approval rate

<b> Alternative hypothesis:</b> loan channel impacts the loan approval rate

<b> Methodology :</b> Using ANOVA test to determine if there is a significant difference between loan default rates across different loan channels. If the test indicates that there is a significant difference, we can use post-hoc tests to determine which specific loan channels have different loan default rates.

In [59]:
data['is_default'] = data['is_default'].astype(int)

In [64]:
formula = 'is_default ~ C(loan_channel)'
model = ols(formula, data=data).fit()
aov_table = sm.stats.anova_lm(model, typ=2)
print(aov_table)

                     sum_sq       df           F         PR(>F)
C(loan_channel)   29.296879      4.0  223.138301  2.286606e-186
Residual         492.191054  14995.0         NaN            NaN


Here, the p-value of the coefficient of loan_channel is less than 0.05, therefore making it significant. Now, I will be using Tukey's Honestly Significant Difference (HSD) test on the data.

In [65]:
mc = sm.stats.multicomp.MultiComparison(data['is_default'], data['loan_channel'])
mc_results = mc.tukeyhsd()
print(mc_results.summary())

       Multiple Comparison of Means - Tukey HSD, FWER=0.05        
    group1         group2     meandiff p-adj  lower  upper  reject
------------------------------------------------------------------
     AFFILIATE          AGENT  -0.0034   0.9 -0.0161 0.0093  False
     AFFILIATE DIRECT_SELLING   0.0004   0.9 -0.0125 0.0133  False
     AFFILIATE     MOBILE_APP  -0.0036   0.9 -0.0165 0.0092  False
     AFFILIATE            WEB   0.1059 0.001  0.0934 0.1185   True
         AGENT DIRECT_SELLING   0.0038   0.9 -0.0091 0.0167  False
         AGENT     MOBILE_APP  -0.0002   0.9 -0.0131 0.0126  False
         AGENT            WEB   0.1093 0.001  0.0968 0.1218   True
DIRECT_SELLING     MOBILE_APP   -0.004   0.9  -0.017  0.009  False
DIRECT_SELLING            WEB   0.1055 0.001  0.0928 0.1182   True
    MOBILE_APP            WEB   0.1095 0.001  0.0969 0.1222   True
------------------------------------------------------------------


If the reject column is true for a pair of loan channels, then we can reject the null hypothesis and conclude that there is a significant difference between the default rates of the loan channels.

Here,the reject column is true for the pairs of loan channels (AFFILIATE, WEB), (AGENT, WEB), (DIRECT_SELLING, WEB), and (MOBILE_APP, WEB). This suggests that the default rate for loans obtained through the WEB channel is significantly different from the default rates for loans obtained through the other channels.

Hence we can reject the null hypothesis, since WEB channel contributes to loan default rate.

Therefore , from the entire analysis, we can say that, interest rates has no effect on the default rate but the type of loan channel does effect the default rate. we have also seen that customer income level has no impact on the loan approval rate.